### Задача 3
---
Склад пункта реализации станков имеет вместимость 20 единиц. Пополнение склада возможно только первого числа каждого месяца. Станки привозят автотранспортом (1 рейс), причем
стоимость рейса складывается из постоянных затрат 50 денежных единиц (ДЕ) и затрат на доставку каждого станка 10 ДЕ. За один рейс (и, следовательно, в месяц) может быть доставлено
не более 5 станков.
Затраты на хранение станка в течение месяца составляют 10 ДЕ.
Ожидаемый спрос на станки приведен в табл. 1. Отсутствие требуемого количества станков
на складе недопустимо.
В начальный момент на складе находится 4 станков.
1. определить план заказов, минимизирующий стоимость;
1. определить границы изменения стоимости хранения станка, в которых найденный план является оптимальным;
1. определить границы изменения постоянных затрат на совершение рейса, в которых найденный план является оптимальным;
1. определить план заказов, минимизирующий стоимость, при условии, что к концу периода на складе должно остаться 3 станков.

Таблица 1: Ожидаемый спрос на станки по месяцам

|  | 1 | 2 | 3 | 4 | 5 | 6 |
|---|:---:|:---:|:---:|:---:|:---:|:---:|
| Спрос, шт. | 0 | 7 | 6 | 3 | 0 | 2 |

In [ ]:
wh_capacity = 20
delivery_cost = 50
unit_delivery_cost = 10
delivery_limit = 5
unit_storage_cost = 10
start_count = 4
expected_demand = [0, 7, 6, 3, 0, 2]

In [ ]:
class NaiveKnapsackSolver:
    def __init__(self, weights, prices, capacity):
        self.q = weights
        self.p = prices
        self.c = capacity
        
    def W(self, stage, state):
        if stage >= len(self.p):
            return (0, None)
        
        best_w = None
        best_u = None
        for u in [0, 1]:
            if u * q[stage] <= state:
                wi = self.p[stage] * self.W(stage + 1, state - q[stage] * u)[0]
                if best_w is None or wi > best_w:
                    best_w = wi
                    best_u = u
        return (best_w, best_u)
    
    def solve(self):
        return self.W(0, self.c)